# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [141]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [5]:
df = pd.read_csv('previsao_de_renda.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [9]:
# Separar os dados em variáveis independentes (X) e variável dependente (y)
X = df.drop('renda', axis=1)
y = df['renda']

# Dividir os dados em conjunto de treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Verificar o tamanho dos conjuntos de treino e teste
print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")

Tamanho do conjunto de treino: 11250
Tamanho do conjunto de teste: 3750


In [123]:
# Separar os dados em variáveis independentes (X) e variável dependente (y)
X = df.drop('renda', axis=1)
y = df['renda']

# Identificar colunas numéricas e categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Criar pré-processadores para numéricas e categóricas, incluindo tratamento de valores faltantes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),  # Preencher valores faltantes com a média
            ('scaler', StandardScaler())  # Escalar os dados
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Preencher valores faltantes com a moda
            ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificar variáveis categóricas
        ]), categorical_features)
    ])

# Criar um pipeline com pré-processamento e modelo Ridge
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('ridge', Ridge())
])

# Definir os parâmetros para GridSearch
param_grid = {
    'ridge__alpha': [0, 0.001, 0.005, 0.01, 0.05, 0.1]
}

# Configurar o GridSearchCV para encontrar o melhor alpha
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Dividir os dados em conjunto de treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Ajustar o modelo aos dados de treino
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e o melhor score
print("Melhor alpha:", grid_search.best_params_['ridge__alpha'])
print("Melhor R^2 no conjunto de validação:", grid_search.best_score_)
print("R^2 no conjunto de teste:", grid_search.score(X_test, y_test))

Melhor alpha: 0.1
Melhor R^2 no conjunto de validação: 0.2551049519421988
R^2 no conjunto de teste: 0.2672790656792706


***Desempenho Comparativo:***
O modelo atual com Ridge Regression tem um R² menor (0.267) em comparação ao modelo anterior do último exercício utilizado na base(0.353). Isso sugere que o modelo anterior estava explicando mais variabilidade da renda em relação ao modelo atual.

___

In [152]:
# Criar o pipeline com o modelo LASSO
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lasso', Lasso())
])

# Definir os parâmetros para GridSearch
param_grid = {
    'lasso__alpha': [0.01, 0.1, 1, 10, 100]
}

# Configurar o GridSearchCV para encontrar o melhor alpha
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Ajustar o modelo aos dados de treino
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e o melhor score
print("Melhor R^2 no conjunto de validação:", grid_search.best_score_)

# Avaliar o modelo no conjunto de teste
y_pred = grid_search.predict(X_test)
r2_test = r2_score(y_test, y_pred)
print("R^2 no conjunto de teste:", r2_test)


Melhor R^2 no conjunto de validação: 0.2572761851751699
R^2 no conjunto de teste: 0.26795606083216295


***Validação:***
A regressão Ridge tem um R2 de validação menor (0.2551) comparado ao LASSO (0.2572). Isso indica que, durante a validação, o modelo LASSO apresentou um melhor desempenho.

***Teste:***
No conjunto de teste, a regressão Ridge apresenta um R2 ligeiramente menor (0.2672) do que o LASSO (0.2679). Portanto, o modelo LASSO se sai melhor no conjunto de teste também.

___

In [158]:
def preprocess_data(X, y):
    # Converter colunas booleanas para numérico
    X['posse_de_veiculo'] = X['posse_de_veiculo'].astype(int)
    X['posse_de_imovel'] = X['posse_de_imovel'].astype(int)
    
    # Excluir colunas desnecessárias se existirem
    cols_to_drop = ['Unnamed: 0', 'id_cliente']
    X = X.drop(columns=[col for col in cols_to_drop if col in X.columns], errors='ignore')
    
    # Converter colunas de X para numérico, forçando erros para NaN
    X = X.apply(pd.to_numeric, errors='coerce')
    
    # Remover colunas que só têm valores NaN
    X = X.dropna(axis=1, how='all')
    
    # Garantir que y é numérico
    y = pd.to_numeric(y, errors='coerce')
    
    # Remover linhas onde a resposta ou os predictores são NaN
    valid_rows = X.notna().all(axis=1) & y.notna()
    X = X[valid_rows]
    y = y[valid_rows]
    
    return X, y

def stepwise_selection(X, y, initial_features=[], threshold_in=0.01, threshold_out=0.05):
    # Pré-processar os dados
    X, y = preprocess_data(X, y)
    
    included = list(initial_features)
    while True:
        changed = False
        
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pvals = pd.Series(index=excluded)
        for new_col in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_col]])).fit()
            new_pvals[new_col] = model.pvalues[new_col]
        best_pval = new_pvals.min()
        if best_pval < threshold_in:
            best_feature = new_pvals.idxmin()
            included.append(best_feature)
            changed = True
        
        # Backward step
        model = sm.OLS(y, sm.add_constant(X[included])).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
        
        if not changed:
            break
    
    return included

# Suponha que seu DataFrame é chamado df e tem uma coluna 'renda' como variável resposta
# Selecione as colunas que serão usadas como variáveis explicativas
X = df.drop(columns=['renda'])
y = df['renda']

# Dividir os dados em treino e teste
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, y, test_size=0.2, random_state=42)

# Usar preprocess_data para limpar os dados de treino e teste
X_train, y_train = preprocess_data(X_train_raw, y_train_raw)
X_test, y_test = preprocess_data(X_test_raw, y_test_raw)

# Criar o pipeline com o modelo LASSO e o pré-processador
pipeline = Pipeline(steps=[
    ('preprocessor', StandardScaler()),  # Normalizar os dados
    ('lasso', Lasso())
])

# Definir os parâmetros para a busca em grid
param_grid = {
    'lasso__alpha': [0.01, 0.1, 1, 10, 100]
}

# Configurar a busca em grid
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train[selected_features], y_train)

# Exibir os melhores parâmetros e o melhor score
print(f"Melhor R^2 no conjunto de validação: {grid_search.best_score_:.4f}")

# Avaliar o modelo no conjunto de teste
y_pred = grid_search.predict(X_test[selected_features])
r2_test = r2_score(y_test, y_pred)
print(f"R^2 no conjunto de teste: {r2_test:.4f}")

Melhor R^2 no conjunto de validação: 0.1314
R^2 no conjunto de teste: 0.1333


***Desempenho em R² no Conjunto de Validação:***
Ridge e LASSO têm valores de R² no conjunto de validação significativamente melhores do que o Stepwise.
O Ridge alcança um R² de 0.2551 e o LASSO um R² de 0.2572, enquanto o Stepwise fica com 0.1314. Isso sugere que os modelos Ridge e LASSO estão conseguindo explicar melhor a variabilidade dos dados no conjunto de validação.

***Desempenho em R² no Conjunto de Teste:***
Novamente, o Ridge e o LASSO apresentam melhor desempenho no conjunto de teste comparado ao Stepwise.
O Ridge tem um R² de 0.2672, o LASSO tem 0.2679, enquanto o Stepwise fica com 0.1333. Isso indica que o Ridge e o LASSO estão proporcionando melhores previsões em dados não vistos.

___

***O modelo LASSO parece ser o melhor de todos, dado que apresenta o melhor desempenho em ambos os conjuntos de dados. Ele consegue capturar a variabilidade dos dados de validação e teste de maneira mais eficaz do que o Ridge e o Stepwise.***

___

In [165]:
# Criar um pipeline com transformação polinomial, escala e modelo Random Forest
pipeline = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Treinar o pipeline
pipeline.fit(X_train, y_train)

# Avaliar o modelo no conjunto de teste
y_pred = pipeline.predict(X_test)
r2_test = r2_score(y_test, y_pred)
print(f"R^2 no conjunto de teste (Pipeline): {r2_test:.4f}")

R^2 no conjunto de teste (Pipeline): 0.1158


**O teste com transformação polinomial, escala e modelo Random Forest se mostrou insatisfatório ao render um R^2 baixo**

In [136]:
# Definir os parâmetros para GridSearch com uma gama mais ampla de alphas
param_grid = {
    'ridge__alpha': [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 10, 100]
}

# Avaliar o modelo no conjunto de teste
y_pred_test = grid_search.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
print("R^2 no conjunto de teste:", r2_test)

R^2 no conjunto de teste: 0.2672790656792706


***Aumentar a gama de valores de alpha ajudou a explorar uma maior variedade de regularizações, o que melhorou o desempenho ao encontrar o melhor parâmetro para o modelo Ridge***

___

In [143]:
# Criar o pipeline com pré-processamento e modelo de árvore de decisão
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', DecisionTreeRegressor(random_state=42))
])

# Ajustar o modelo aos dados de treino
pipeline.fit(X_train, y_train)

# Avaliar o modelo no conjunto de teste
y_pred = pipeline.predict(X_test)
r2_test = r2_score(y_test, y_pred)
print(f"R^2 no conjunto de teste com árvore de decisão: {r2_test:.4f}")

R^2 no conjunto de teste com árvore de decisão: 0.0464


***O R^2 foi o menor de todos os modelos o que demonstra que o modelo de árvore de decisão não teve um bom desempenho***

In [180]:
# Definir os parâmetros para GridSearch
param_grid = {
    'tree__max_depth': [None, 10, 20, 30, 40, 50],
    'tree__min_samples_split': [2, 5, 10],
    'tree__min_samples_leaf': [1, 2, 4]
}

# Criar o pipeline com pré-processamento e modelo de árvore de decisão
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', DecisionTreeRegressor(random_state=42))
])

# Configurar o GridSearchCV para encontrar os melhores parâmetros
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Dividir os dados em conjunto de treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Ajustar o modelo aos dados de treino
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e o melhor score
print("Melhor R^2 no conjunto de validação:", grid_search.best_score_)

# Avaliar o modelo no conjunto de teste
y_pred = grid_search.predict(X_test)
r2_test = r2_score(y_test, y_pred)
print(f"R^2 no conjunto de teste com árvore de decisão otimizada: {r2_test:.4f}")

Melhor R^2 no conjunto de validação: 0.29775147309310707
R^2 no conjunto de teste com árvore de decisão otimizada: 0.2674


***A árvore de decisão otimizada conseguiu um R² de 0.2674 no conjunto de teste, o que é uma melhora em relação aos modelos anteriores. Isso indica que a árvore de decisão, com os parâmetros otimizados, conseguiu ficar em segundo lugar como modelo mais eficiente, perdendo apenas para o modelo LASSO.***